<a href="https://colab.research.google.com/github/LSC10/test/blob/main/RetrievalCorpusMindline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet --upgrade \
    unstructured \
    langchain \
    langchain-community \
    langchain-openai \
    langchain-text-splitters \
    langgraph \
    beautifulsoup4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━

Loading the data: mindline, wysa, intellect, other_resources

In [ ]:
import os, getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"


Enter your OpenAI API key: ··········
Enter your LangSmith API key: ··········


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/Shareddrives/MOHT | Internal/Mindline Datasets/"


intellect_resources_urls.csv  other_resources_urls.csv
mindline_url.csv	      wysa_resources_urls.csv


In [ ]:
import pandas as pd

base_path = "/content/drive/Shareddrives/MOHT | Internal/Mindline Datasets/"

# Load each dataset
mindline_df = pd.read_csv(base_path + "mindline_url.csv")
wysa_df = pd.read_csv(base_path + "wysa_resources_urls.csv")
intellect_df = pd.read_csv(base_path + "intellect_resources_urls.csv")
other_df = pd.read_csv(base_path + "other_resources_urls.csv")


In [ ]:
for name, df in zip(
    ["Mindline", "Wysa", "Intellect", "Other"],
    [mindline_df, wysa_df, intellect_df, other_df]
):
    print(f"\n==== {name} ====")
    print(df.head(3))
    print(df.columns)
    print(f"{name} rows: {len(df)}")



==== Mindline ====
                                                 url
0  https://www.mindline.sg/youth/resource-group/s...
1  https://www.mindline.sg/youth/resource-group/h...
2  https://www.mindline.sg/youth/resource-group/a...
Index(['url'], dtype='object')
Mindline rows: 446

==== Wysa ====
               Tool_id                                             source  \
0  understand_yourself  https://mindline.sg/workspace?wysa_tool_id=und...   
1       build_strength  https://mindline.sg/workspace?wysa_tool_id=bui...   
2         hold_another  https://mindline.sg/workspace?wysa_tool_id=hol...   

                                        page_content  
0  This wysa exercise is called Understand Yourse...  
1  This wysa exercise is called Build Strength. T...  
2  This wysa exercise is called Hold Another. Thi...  
Index(['Tool_id', 'source', 'page_content'], dtype='object')
Wysa rows: 63

==== Intellect ====
                          Tool_id  \
0      Intellect: Staying Focused   
1  

In [ ]:
mindline_df.columns
wysa_df.columns
intellect_df.columns
other_df.columns


Index(['Tool_id', 'source', 'page_content'], dtype='object')

Web scraping the mindline resources using the urls. Wysa, intellect, and other resources do not need to be web scraped because they already have page_content provided.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader




In [ ]:
def web_scrape(urls):
  # Initialize loader WITHOUT specifying bs4_strainer (scrape all visible text)
  loader = WebBaseLoader(web_paths=urls)

  # Load documents
  docs = loader.load()

  # Print summary of results
  print(f"Scraped {len(docs)} documents\n")

  # Preview the first document's content
  if docs:
      print("--- First Document Preview ---\n")
      print(docs[0].page_content[:1500])  # Print the first 1500 characters
  else:
      print("No content was scraped.")
  return docs



In [ ]:
urls = mindline_df["url"].tolist()


In [ ]:
web_scrape(urls[150:155])

Scraped 5 documents

--- First Document Preview ---

Tips for building resilience - ArticleGet Help Now!ArticlesTips for building resilienceHaving resilienceWhy is it that sometimes people go through really rough times and still bounce back? The difference is that those who bounce back are using resilience. The skill of resilience is something that can be learned. Resilience - the ability to adapt well in the face of hard times - is what makes some people seem like they have "got bounce" while others don't.Tips to build resilience1.Get together with your friends and parents - ask questions and listen to the answers2.Cut yourself some slack - go easy on yourself when you are met with stresses3.Create a hassle-free zone - not to keep everyone out, but to have a space free from stresses4.Map out a routine and stick to it - during times of stress, don't forget the routines that give you comfort5.Take care of yourself - physically, mentally and spirituallyTips to build resilience6.Take cont

[Document(metadata={'source': 'https://www.mindline.sg/youth/article/tips-for-building-resilience?type=interest', 'title': 'Tips for building resilience - Article', 'language': 'en'}, page_content='Tips for building resilience - ArticleGet Help Now!ArticlesTips for building resilienceHaving resilienceWhy is it that sometimes people go through really rough times and still bounce back? The difference is that those who bounce back are using resilience. The skill of resilience is something that can be learned. Resilience - the ability to adapt well in the face of hard times - is what makes some people seem like they have "got bounce" while others don\'t.Tips to build resilience1.Get together with your friends and parents - ask questions and listen to the answers2.Cut yourself some slack - go easy on yourself when you are met with stresses3.Create a hassle-free zone - not to keep everyone out, but to have a space free from stresses4.Map out a routine and stick to it - during times of stress

In [ ]:
mindline_df = pd.DataFrame([{
    "source": doc.metadata.get("source", ""),
    "page_content": doc.page_content
} for doc in docs])

# Optional: Preview result
print(mindline_df.head(2))

                                              source  \
0  https://www.mindline.sg/youth/resource-group/m...   

                          page_content  
0  Confidence and Self-Esteem - Myself  


Combining All Four Resources (mindline, wysa, intellect, and other) into one corpus

In [ ]:
from langchain_core.documents import Document

def df_to_documents(df, source_col="source", content_col="page_content"):
    return [
        Document(page_content=row[content_col], metadata={"source": row[source_col]})
        for _, row in df.iterrows()
    ]

mindline_docs = df_to_documents(mindline_df)
wysa_docs = df_to_documents(wysa_df)
intellect_docs = df_to_documents(intellect_df)
other_docs = df_to_documents(other_df)


In [ ]:
all_docs = mindline_docs + wysa_docs + intellect_docs + other_docs
print(f"Total combined documents: {len(all_docs)}")


Split corpus into chunks for better embedding and retrieval

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)

all_chunks = text_splitter.split_documents(all_docs)
print(f"Total chunks after splitting: {len(all_chunks)}")


In [ ]:
# store in vector store
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore  # For quick testing; FAISS is better for production

embeddings = OpenAIEmbeddings()
vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_documents(all_chunks)


In [ ]:
query = "How do I deal with anxiety from school stress?"
results = vector_store.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n--- Document {i+1} ---")
    print(doc.page_content[:500])
    print("Source:", doc.metadata.get("source"))
